In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import statistics

In [40]:
bids_df = pd.read_csv('data/bids.csv')
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [41]:
bids_df.head()

,bidder_id,auction,merchandise,device,time,country,ip,url
0,6b18a5b1eb756ed0968e726077ac6f574a163,hfwg0,sporting goods,phone4261,9631916842105263,cz,58.66.142.81,kiepbuv7y92mx3t
1,94587f4554fd3b271147490c35a27549f89f7,22iq4,jewelry,phone738,9631916842105263,ru,95.161.165.11,4sqivcoh12vctv7
2,cfe767f0e4cd7c62515326a795a661c422dcc,w69k8,mobile,phone3459,9631916842105263,lt,75.233.227.15,ae97emyjh9trong
3,73fe2fb4f9d657de35ccf3ad4fc8c1603e43c,k8pyh,jewelry,phone3459,9631916842105263,sa,49.252.224.185,26mecamx4l5vhiq
4,39ce4ad64b91b87b9f6e31064644f7a011b04,c1ok3,office equipment,phone3311,9631916842105263,tr,54.83.204.113,ae97emyjh9trong


In [42]:
train_df.shape

(2013, 4)

In [43]:
bids_df.nunique()

bidder_id         6614
auction          15051
merchandise         10
device            7351
time            776529
country            199
ip             2303991
url            1786351
dtype: int64

In [44]:
merch_ohe = pd.get_dummies(bids_df['merchandise'])
merch_ohe.head()

,auto parts,books and music,clothing,computers,furniture,home goods,jewelry,mobile,office equipment,sporting goods
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0


In [45]:
bids_merch_df = pd.concat([bids_df['bidder_id'], merch_ohe], axis=1)
bids_merch_df 
# 'auto parts', 'books and music', 'computers', 'furniture', 'home goods', 'jewelry', 'mobile', 'office equipment', 'sporting goods'

,bidder_id,auto parts,books and music,clothing,computers,furniture,home goods,jewelry,mobile,office equipment,sporting goods
0,6b18a5b1eb756ed0968e726077ac6f574a163,0,0,0,0,0,0,0,0,0,1
1,94587f4554fd3b271147490c35a27549f89f7,0,0,0,0,0,0,1,0,0,0
2,cfe767f0e4cd7c62515326a795a661c422dcc,0,0,0,0,0,0,0,1,0,0
3,73fe2fb4f9d657de35ccf3ad4fc8c1603e43c,0,0,0,0,0,0,1,0,0,0
4,39ce4ad64b91b87b9f6e31064644f7a011b04,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
7656329,36638ed048f5bf872bf6821ae92c12ddc4b8f,0,0,0,0,0,1,0,0,0,0
7656330,e51acc084941dbfd132b463cc4c93f4e0650d,0,0,0,0,0,0,1,0,0,0
7656331,0fd0eb00687f477f04f738bc1180954522540,0,0,0,0,0,0,1,0,0,0
7656332,3711dbf7b1d24741363721edae3d53ae0ce40,0,0,0,0,0,0,1,0,0,0


In [46]:
bids_merch_sum = bids_merch_df.groupby('bidder_id').sum().reset_index()
bids_merch_sum.head()

,bidder_id,auto parts,books and music,clothing,computers,furniture,home goods,jewelry,mobile,office equipment,sporting goods
0,0013f9b8d1f462df4462e1c1216e441ba6e18,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
2,002828f800c5132e297cfe3d44fbde9aeac51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,00402f50c4086f320cb6bf94f04462ea32441,0.0,0.0,0.0,0.0,0.0,0.0,249.0,0.0,0.0,0.0
4,0040cea6b93afd86768c365d89513ffb7c0ba,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0


In [47]:
bids_df_count = bids_df.groupby('bidder_id').count()
bids_df_count.shape

(6614, 7)

In [48]:
bids_df_unique_count = bids_df.groupby('bidder_id').nunique().drop(columns='bidder_id')

In [49]:
bids_df_unique_count = bids_df_unique_count.reset_index()
bids_df_unique_count.head()

,bidder_id,auction,merchandise,device,time,country,ip,url
0,0013f9b8d1f462df4462e1c1216e441ba6e18,6,1,7,8,2,8,4
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7,1,7,7,4,7,4
2,002828f800c5132e297cfe3d44fbde9aeac51,2,1,1,2,1,2,1
3,00402f50c4086f320cb6bf94f04462ea32441,89,1,99,249,21,151,98
4,0040cea6b93afd86768c365d89513ffb7c0ba,23,1,21,31,6,28,4


In [50]:
time_df = bids_df[['bidder_id', 'time']]
time_df['time_6'] = time_df['time'].astype(str).str.slice(stop=6)
time_df['time_5'] = time_df['time'].astype(str).str.slice(stop=5)
time_df['time_4'] = time_df['time'].astype(str).str.slice(stop=4)

<ipython-input-50-00ddfc88001f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df['time_6'] = time_df['time'].astype(str).str.slice(stop=6)
<ipython-input-50-00ddfc88001f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df['time_5'] = time_df['time'].astype(str).str.slice(stop=5)
<ipython-input-50-00ddfc88001f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [51]:
time_final_df = time_df.groupby('bidder_id').nunique().drop(columns=['bidder_id','time'])
time_final_df = time_final_df.reset_index()
time_final_df

,bidder_id,time_6,time_5,time_4
0,0013f9b8d1f462df4462e1c1216e441ba6e18,8,8,6
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7,7,6
2,002828f800c5132e297cfe3d44fbde9aeac51,2,2,2
3,00402f50c4086f320cb6bf94f04462ea32441,204,101,14
4,0040cea6b93afd86768c365d89513ffb7c0ba,28,27,18
...,...,...,...,...
6609,ffd49be672b1ba493b07dccf29311045c5392,144,83,14
6610,ffdd8ed91a683b0f2a8237798ebe9214e3c43,5,4,3
6611,ffde8717e9a547d536a4e9c3f44782829c021,24,24,16
6612,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,3,2,2


In [52]:
bids_df_unique_count.shape

(6614, 8)

In [53]:
join_result = train_df.merge(bids_df_unique_count, how='left')
join_result.shape

(2013, 11)

In [54]:
join_result2 = join_result.merge(bids_merch_sum, how='left')
join_result2.shape

(2013, 21)

In [55]:
join_result3 = join_result2.merge(time_final_df, how='left')
join_result3.shape

(2013, 24)

In [56]:
import numpy as np
count = bids_df_count[['url']].reset_index()
count = count.rename(columns={'url': 'num_bids'})
count['num_bids_log'] = (count['num_bids']+1).transform(np.log)
count.head()

,bidder_id,num_bids,num_bids_log
0,0013f9b8d1f462df4462e1c1216e441ba6e18,8,2.197225
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,7,2.079442
2,002828f800c5132e297cfe3d44fbde9aeac51,2,1.098612
3,00402f50c4086f320cb6bf94f04462ea32441,249,5.521461
4,0040cea6b93afd86768c365d89513ffb7c0ba,31,3.465736


In [57]:
outlier_train_df = join_result3.merge(count, how='left')
outlier_train_df.shape

(2013, 26)

In [58]:
outlier_train_df = outlier_train_df.fillna(0)
outlier_train_df

,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,ip,...,home goods,jewelry,mobile,office equipment,sporting goods,time_6,time_5,time_4,num_bids,num_bids_log
0,3758013f9a2c4f9d8670e709d9e8beaba6f57,32a6f444ff094840b58be6bb6db6a04fe4f85,4f2ed7fc454d535ecba88bc7f854925dfe4ab,0.0,25.0,1.0,30.0,50.0,8.0,37.0,...,0.0,0.0,50.0,0.0,0.0,46.0,33.0,13.0,50.0,3.931826
1,8a12c97c9dac9ef7fcbbceaf7522447cf6e49,eb0ae407d20ab19351c5271abe4ca12ce84c0,1c73853ef410e4095382f8d6ddeb0b9d22bb0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.693147
2,27b3a0fecc63bf830bc44961c90f330308364,87dbae72fd15fa13f14217e47f32fa0ea9417,c4bbeba55def4b3861ac24cc5efd82b886b9b,0.0,2.0,1.0,1.0,3.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,3.0,1.386294
3,2a2b18bbdfa736637911cf407799b409112a2,40a0df1d610ef30a8f01d48198ab60bee51e2,021918f1b3fc828970b230eaf4d49f8a93b95,0.0,33.0,1.0,22.0,61.0,4.0,34.0,...,61.0,0.0,0.0,0.0,0.0,55.0,38.0,13.0,61.0,4.127134
4,3fdc7c645e5e131c05f1ae112dbae407523c5,3d20d9941901359e5fed3dfc0c5bf45dfd968,d6331a777cabb7c5df46c6a0377136b904614,0.0,377.0,1.0,462.0,3177.0,72.0,2318.0,...,0.0,3184.0,0.0,0.0,0.0,1720.0,275.0,30.0,3184.0,8.066208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,6c882acd41865f09edbef0075d0224feeb246,cc53fc05a77105352f5c52ad45d6dc16d6cf1,96507e44143695e8df908908e40d7922caebd,0.0,68.0,1.0,125.0,296.0,27.0,210.0,...,0.0,297.0,0.0,0.0,0.0,228.0,101.0,14.0,297.0,5.697093
2009,0bbbc9ab5b0f4117de20eec945c10b027c812,2b9e6e1815d8a59fc5533dc90cd376bb949c8,5182e89f3ca6a1614321f8d9b288c111013de,0.0,7.0,1.0,5.0,14.0,1.0,7.0,...,0.0,14.0,0.0,0.0,0.0,8.0,5.0,4.0,14.0,2.708050
2010,db8d65fd2c6934e9c3448398c3bd442f52a1a,d0056d416ad19db0b523070857ab165362475,df1ec221c46b2995563aceac9c60da8e68746,0.0,23.0,1.0,15.0,26.0,8.0,19.0,...,0.0,0.0,26.0,0.0,0.0,22.0,15.0,10.0,26.0,3.295837
2011,e2697e6c718d4bccf18f647c5284b6973bfc4,101bcafedae6b0f5cac5a1f7aab0990764ff7,479b26a0b6826749981ba0d7946c04e23b6ab,0.0,28.0,1.0,3.0,54.0,4.0,5.0,...,0.0,54.0,0.0,0.0,0.0,15.0,6.0,4.0,54.0,4.007333


In [59]:
new_train_df = outlier_train_df.loc[((outlier_train_df.outcome == 1) & ((outlier_train_df.num_bids) > 1)) | (outlier_train_df.outcome == 0)]
# new_train_df = new_train_df.reset_index().drop(columns=['index'])
new_train_df = new_train_df .reset_index()
new_train_df  = new_train_df .drop(columns=['index'])
new_train_df 

,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,ip,...,home goods,jewelry,mobile,office equipment,sporting goods,time_6,time_5,time_4,num_bids,num_bids_log
0,3758013f9a2c4f9d8670e709d9e8beaba6f57,32a6f444ff094840b58be6bb6db6a04fe4f85,4f2ed7fc454d535ecba88bc7f854925dfe4ab,0.0,25.0,1.0,30.0,50.0,8.0,37.0,...,0.0,0.0,50.0,0.0,0.0,46.0,33.0,13.0,50.0,3.931826
1,8a12c97c9dac9ef7fcbbceaf7522447cf6e49,eb0ae407d20ab19351c5271abe4ca12ce84c0,1c73853ef410e4095382f8d6ddeb0b9d22bb0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.693147
2,27b3a0fecc63bf830bc44961c90f330308364,87dbae72fd15fa13f14217e47f32fa0ea9417,c4bbeba55def4b3861ac24cc5efd82b886b9b,0.0,2.0,1.0,1.0,3.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,3.0,1.386294
3,2a2b18bbdfa736637911cf407799b409112a2,40a0df1d610ef30a8f01d48198ab60bee51e2,021918f1b3fc828970b230eaf4d49f8a93b95,0.0,33.0,1.0,22.0,61.0,4.0,34.0,...,61.0,0.0,0.0,0.0,0.0,55.0,38.0,13.0,61.0,4.127134
4,3fdc7c645e5e131c05f1ae112dbae407523c5,3d20d9941901359e5fed3dfc0c5bf45dfd968,d6331a777cabb7c5df46c6a0377136b904614,0.0,377.0,1.0,462.0,3177.0,72.0,2318.0,...,0.0,3184.0,0.0,0.0,0.0,1720.0,275.0,30.0,3184.0,8.066208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,6c882acd41865f09edbef0075d0224feeb246,cc53fc05a77105352f5c52ad45d6dc16d6cf1,96507e44143695e8df908908e40d7922caebd,0.0,68.0,1.0,125.0,296.0,27.0,210.0,...,0.0,297.0,0.0,0.0,0.0,228.0,101.0,14.0,297.0,5.697093
2004,0bbbc9ab5b0f4117de20eec945c10b027c812,2b9e6e1815d8a59fc5533dc90cd376bb949c8,5182e89f3ca6a1614321f8d9b288c111013de,0.0,7.0,1.0,5.0,14.0,1.0,7.0,...,0.0,14.0,0.0,0.0,0.0,8.0,5.0,4.0,14.0,2.708050
2005,db8d65fd2c6934e9c3448398c3bd442f52a1a,d0056d416ad19db0b523070857ab165362475,df1ec221c46b2995563aceac9c60da8e68746,0.0,23.0,1.0,15.0,26.0,8.0,19.0,...,0.0,0.0,26.0,0.0,0.0,22.0,15.0,10.0,26.0,3.295837
2006,e2697e6c718d4bccf18f647c5284b6973bfc4,101bcafedae6b0f5cac5a1f7aab0990764ff7,479b26a0b6826749981ba0d7946c04e23b6ab,0.0,28.0,1.0,3.0,54.0,4.0,5.0,...,0.0,54.0,0.0,0.0,0.0,15.0,6.0,4.0,54.0,4.007333


In [60]:
# k fold cross validation 
import xgboost as xgb
from sklearn.metrics import roc_curve, auc

In [61]:
y_xgb = new_train_df ['outcome']
X_xgb = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome'])

kf = KFold(n_splits=20)
model_xgb = xgb.XGBClassifier(random_state=5)

kfold_auc_xgb = []

for train_index, test_index in kf.split(X_xgb):
    X_train, X_test = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
    y_train, y_test = y_xgb.iloc[train_index], y_xgb.iloc[test_index]
    
    model_xgb.fit(X_train, y_train)
    y_predict_xgb = model_xgb.predict_proba(X_test)[:,1]
    
    fpr, tpr, thresholds = roc_curve(y_test, y_predict_xgb)
    auc_num = auc(fpr, tpr)
    kfold_auc_xgb.append(auc_num)

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:42:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:42:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:42:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:42:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [62]:
avg_auc_xgb = statistics.mean(kfold_auc_xgb)
avg_auc_xgb

0.9182539717590318

In [63]:
from sklearn.ensemble import RandomForestClassifier
y_rand_forest = new_train_df['outcome']
X_rand_forest = new_train_df[['auction','merchandise', 'device', 'time', 'country','time_6','time_5','time_4','num_bids','num_bids_log']]

kf = KFold(n_splits=20)
model_random_forest = RandomForestClassifier(random_state=5)

kfold_auc_random_forest = []

for train_index, test_index in kf.split(X_rand_forest):
    X_train, X_test = X_rand_forest.iloc[train_index], X_rand_forest.iloc[test_index]
    y_train, y_test = y_rand_forest.iloc[train_index], y_rand_forest.iloc[test_index]
    
    model_random_forest.fit(X_train, y_train)
    y_predict_random_forest = model_random_forest.predict_proba(X_test)[:,1]
    
    fpr, tpr, thresholds = roc_curve(y_test, y_predict_random_forest)
    auc_num = auc(fpr, tpr)
    kfold_auc_random_forest.append(auc_num)

In [64]:
avg_auc_random_forest = statistics.mean(kfold_auc_random_forest)
avg_auc_random_forest

0.9244745988616135

In [65]:
from catboost import CatBoostClassifier

y_catboost = new_train_df['outcome']
X_catboost = new_train_df[['auction','merchandise', 'device', 'time', 'country','time_6','time_5','time_4','num_bids','num_bids_log']]

kf = KFold(n_splits=20)
model_catboost = CatBoostClassifier(random_seed=5)

kfold_auc_catboost = []

for train_index, test_index in kf.split(X_catboost):
    X_train, X_test = X_catboost.iloc[train_index], X_catboost.iloc[test_index]
    y_train, y_test = y_catboost.iloc[train_index], y_catboost.iloc[test_index]
    
    model_catboost.fit(X_train, y_train)
    y_predict_catboost = model_catboost.predict_proba(X_test)[:,1]
    
    fpr, tpr, thresholds = roc_curve(y_test, y_predict_catboost)
    auc_num = auc(fpr, tpr)
    kfold_auc_catboost.append(auc_num)

Learning rate set to 0.013572
0:	learn: 0.6641860	total: 33.7ms	remaining: 33.6s
1:	learn: 0.6329677	total: 36.8ms	remaining: 18.4s
2:	learn: 0.6059464	total: 39.8ms	remaining: 13.2s
3:	learn: 0.5854474	total: 42.7ms	remaining: 10.6s
4:	learn: 0.5614666	total: 47.4ms	remaining: 9.44s
5:	learn: 0.5402186	total: 51.1ms	remaining: 8.47s
6:	learn: 0.5193969	total: 54.1ms	remaining: 7.67s
7:	learn: 0.4985437	total: 57ms	remaining: 7.07s
8:	learn: 0.4839096	total: 60.1ms	remaining: 6.61s
9:	learn: 0.4634821	total: 63.2ms	remaining: 6.26s
10:	learn: 0.4466921	total: 67.2ms	remaining: 6.04s
11:	learn: 0.4344756	total: 70.2ms	remaining: 5.78s
12:	learn: 0.4192922	total: 73.3ms	remaining: 5.56s
13:	learn: 0.4041864	total: 76.3ms	remaining: 5.37s
14:	learn: 0.3909329	total: 79.7ms	remaining: 5.24s
15:	learn: 0.3767754	total: 82.7ms	remaining: 5.08s
16:	learn: 0.3690669	total: 85.8ms	remaining: 4.96s
17:	learn: 0.3566713	total: 88.9ms	remaining: 4.85s
18:	learn: 0.3493475	total: 91.8ms	remaining: 

In [66]:
avg_auc_catboost = statistics.mean(kfold_auc_catboost)
avg_auc_catboost

0.9411186741220927

In [67]:
# random forest and catboost
y = new_train_df['outcome']
X = new_train_df[['auction','merchandise', 'device', 'time', 'country','time_6','time_5','time_4','num_bids','num_bids_log']]

kf = KFold(n_splits=20)
model_catboost = CatBoostClassifier(random_seed=5)
model_random_forest = RandomForestClassifier(random_state=5)
model_xgb = xgb.XGBClassifier(random_state=5)

kfold_auc_avg = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model_catboost.fit(X_train, y_train)
    y_predict_catboost = model_catboost.predict_proba(X_test)[:,1]

    model_random_forest.fit(X_train, y_train)
    y_predict_random_forest = model_random_forest.predict_proba(X_test)[:,1]
    
    model_xgb.fit(X_train, y_train)
    y_predict_xgb = model_xgb.predict_proba(X_test)[:, 1]
    
    y_final_pred = y_predict_catboost*0.8 + y_predict_random_forest*0.1 + y_predict_xgb*0.1
    
    fpr, tpr, thresholds = roc_curve(y_test, y_final_pred)
    auc_num = auc(fpr, tpr)
    kfold_auc_avg.append(auc_num)

Learning rate set to 0.013572
0:	learn: 0.6641860	total: 5.68ms	remaining: 5.68s
1:	learn: 0.6329677	total: 9.56ms	remaining: 4.77s
2:	learn: 0.6059464	total: 13.5ms	remaining: 4.47s
3:	learn: 0.5854474	total: 16.7ms	remaining: 4.17s
4:	learn: 0.5614666	total: 20.7ms	remaining: 4.13s
5:	learn: 0.5402186	total: 24.2ms	remaining: 4s
6:	learn: 0.5193969	total: 27.7ms	remaining: 3.94s
7:	learn: 0.4985437	total: 31.6ms	remaining: 3.92s
8:	learn: 0.4839096	total: 35.6ms	remaining: 3.92s
9:	learn: 0.4634821	total: 40.8ms	remaining: 4.04s
10:	learn: 0.4466921	total: 49.7ms	remaining: 4.47s
11:	learn: 0.4344756	total: 55.8ms	remaining: 4.6s
12:	learn: 0.4192922	total: 62.2ms	remaining: 4.72s
13:	learn: 0.4041864	total: 69.5ms	remaining: 4.9s
14:	learn: 0.3909329	total: 74.5ms	remaining: 4.89s
15:	learn: 0.3767754	total: 79.7ms	remaining: 4.9s
16:	learn: 0.3690669	total: 83.2ms	remaining: 4.81s
17:	learn: 0.3566713	total: 86.9ms	remaining: 4.74s
18:	learn: 0.3493475	total: 90.7ms	remaining: 4.68

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:43:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.013572
0:	learn: 0.6627618	total: 6.16ms	remaining: 6.16s
1:	learn: 0.6297866	total: 10.4ms	remaining: 5.19s
2:	learn: 0.6015070	total: 14.6ms	remaining: 4.87s
3:	learn: 0.5808294	total: 18.7ms	remaining: 4.66s
4:	learn: 0.5556280	total: 22.8ms	remaining: 4.53s
5:	learn: 0.5346846	total: 26.6ms	remaining: 4.41s
6:	learn: 0.5131282	total: 31ms	remaining: 4.4s
7:	learn: 0.4914612	total: 35ms	remaining: 4.34s
8:	learn: 0.4769428	total: 39ms	remaining: 4.29s
9:	learn: 0.4553708	total: 43.2ms	remaining: 4.27s
10:	learn: 0.4379908	total: 47.1ms	remaining: 4.24s
11:	learn: 0.4256770	total: 51.1ms	remaining: 4.21s
12:	learn: 0.4105816	total: 55ms	remaining: 4.18s
13:

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:43:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.013572
0:	learn: 0.6641047	total: 5.19ms	remaining: 5.19s
1:	learn: 0.6327859	total: 9.41ms	remaining: 4.7s
2:	learn: 0.6052587	total: 14.9ms	remaining: 4.95s
3:	learn: 0.5840755	total: 19.9ms	remaining: 4.96s
4:	learn: 0.5600926	total: 25.5ms	remaining: 5.08s
5:	learn: 0.5379546	total: 30.1ms	remaining: 4.98s
6:	learn: 0.5170590	total: 33.8ms	remaining: 4.79s
7:	learn: 0.4961154	total: 37.5ms	remaining: 4.65s
8:	learn: 0.4813369	total: 42.3ms	remaining: 4.65s
9:	learn: 0.4608441	total: 46.5ms	remaining: 4.61s
10:	learn: 0.4426409	total: 50.5ms	remaining: 4.54s
11:	learn: 0.4304376	total: 54.5ms	remaining: 4.49s
12:	learn: 0.4153069	total: 58.8ms	remaining: 4

...

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:45:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.013575
0:	learn: 0.6640861	total: 4.42ms	remaining: 4.42s
1:	learn: 0.6324322	total: 9.04ms	remaining: 4.51s
2:	learn: 0.6048513	total: 13.4ms	remaining: 4.44s
3:	learn: 0.5838169	total: 17.6ms	remaining: 4.37s
4:	learn: 0.5597376	total: 22ms	remaining: 4.37s
5:	learn: 0.5379418	total: 26.3ms	remaining: 4.35s
6:	learn: 0.5208019	total: 31.4ms	remaining: 4.46s
7:	learn: 0.4997155	total: 38.5ms	remaining: 4.77s
8:	learn: 0.4844475	total: 42.2ms	remaining: 4.64s
9:	learn: 0.4636299	total: 45.9ms	remaining: 4.54s
10:	learn: 0.4457368	total: 49.9ms	remaining: 4.49s
11:	learn: 0.4328253	total: 53.6ms	remaining: 4.41s
12:	learn: 0.4177790	total: 57.4ms	remaining: 4.

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:45:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.013575
0:	learn: 0.6624780	total: 6.13ms	remaining: 6.12s
1:	learn: 0.6337220	total: 10.3ms	remaining: 5.14s
2:	learn: 0.6100879	total: 14.5ms	remaining: 4.82s
3:	learn: 0.5837386	total: 18.7ms	remaining: 4.66s
4:	learn: 0.5582784	total: 23.1ms	remaining: 4.6s
5:	learn: 0.5367115	total: 28.1ms	remaining: 4.65s
6:	learn: 0.5155014	total: 33.6ms	remaining: 4.76s
7:	learn: 0.4996649	total: 37.7ms	remaining: 4.68s
8:	learn: 0.4793601	total: 41.4ms	remaining: 4.55s
9:	learn: 0.4634242	total: 45.1ms	remaining: 4.46s
10:	learn: 0.4468762	total: 48.9ms	remaining: 4.39s
11:	learn: 0.4307422	total: 53.1ms	remaining: 4.37s
12:	learn: 0.4145825	total: 56.9ms	remaining: 4

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [68]:
avg_auc_avg = statistics.mean(kfold_auc_avg)
avg_auc_avg

0.9401128830056023

In [69]:
# AFTER CONFIRMING STEP# 



In [86]:
new_train_df.columns

Index(['bidder_id', 'payment_account', 'address', 'outcome', 'auction',
       'merchandise', 'device', 'time', 'country', 'ip', 'url', 'auto parts',
       'books and music', 'clothing', 'computers', 'furniture', 'home goods',
       'jewelry', 'mobile', 'office equipment', 'sporting goods', 'time_6',
       'time_5', 'time_4', 'num_bids', 'num_bids_log'],
      dtype='object')

In [87]:
test_join1 = test_df.merge(bids_df_unique_count, how='left')
test_join2 = test_join1.merge(bids_merch_sum, how='left')
test_join3 = test_join2.merge(time_final_df, how='left')
final_test = test_join3.merge(count, how='left')

In [88]:
final_test.columns

Index(['bidder_id', 'payment_account', 'address', 'auction', 'merchandise',
       'device', 'time', 'country', 'ip', 'url', 'auto parts',
       'books and music', 'clothing', 'computers', 'furniture', 'home goods',
       'jewelry', 'mobile', 'office equipment', 'sporting goods', 'time_6',
       'time_5', 'time_4', 'num_bids', 'num_bids_log'],
      dtype='object')

In [73]:
final_test = final_test.fillna(0)

In [74]:
# y_final = new_train_df['outcome']
# X_final = new_train_df.drop(columns=['bidder_id','payment_account','address','outcome'])

# model_xgb = xgb.XGBClassifier(random_state=5)
# model_xgb.fit(X_final1,y_final1)

# X_test_xgb = final_test.drop(columns=['bidder_id','payment_account','address'])
# y_predict_xgb_final = model_xgb.predict_proba(X_test_xgb)[:,1]

In [75]:
y_final = new_train_df['outcome']
X_final = new_train_df[['auction','merchandise', 'device', 'time', 'country', 'time_6','time_5','time_4','num_bids', 'num_bids_log']]

model_catboost = CatBoostClassifier(random_seed=5)
model_random_forest = RandomForestClassifier(random_state=5)
model_xgb = xgb.XGBClassifier(random_state=5)

model_catboost.fit(X_final, y_final)
model_random_forest.fit(X_final, y_final)
model_xgb.fit(X_final, y_final)

X_test = final_test[['auction','merchandise', 'device', 'time', 'country','time_6','time_5','time_4','num_bids', 'num_bids_log']]
y_predict_catboost = model_catboost.predict_proba(X_test)[:,1]
y_predict_random_forest= model_random_forest.predict_proba(X_test)[:,1]
y_predict_xgb = model_xgb.predict_proba(X_test)[:, 1]

Learning rate set to 0.013874
0:	learn: 0.6632724	total: 5.08ms	remaining: 5.07s
1:	learn: 0.6311106	total: 8.87ms	remaining: 4.43s
2:	learn: 0.6029896	total: 12.5ms	remaining: 4.16s
3:	learn: 0.5817826	total: 16.8ms	remaining: 4.18s
4:	learn: 0.5570344	total: 20.3ms	remaining: 4.04s
5:	learn: 0.5346817	total: 24ms	remaining: 3.98s
6:	learn: 0.5133480	total: 27.5ms	remaining: 3.9s
7:	learn: 0.4920728	total: 31.4ms	remaining: 3.89s
8:	learn: 0.4770796	total: 35.1ms	remaining: 3.86s
9:	learn: 0.4561527	total: 40.4ms	remaining: 4s
10:	learn: 0.4380108	total: 50.6ms	remaining: 4.55s
11:	learn: 0.4250813	total: 55.7ms	remaining: 4.58s
12:	learn: 0.4103589	total: 60.9ms	remaining: 4.62s
13:	learn: 0.3960547	total: 66ms	remaining: 4.65s
14:	learn: 0.3826770	total: 69.5ms	remaining: 4.56s
15:	learn: 0.3684847	total: 73.1ms	remaining: 4.5s
16:	learn: 0.3609469	total: 81.1ms	remaining: 4.69s
17:	learn: 0.3482983	total: 84.7ms	remaining: 4.62s
18:	learn: 0.3384855	total: 88.2ms	remaining: 4.55s
1

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:45:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [76]:
print(len(y_predict_catboost))
print(len(y_predict_random_forest))
print(len(y_predict_xgb))

4700
4700
4700


In [90]:
y_final_try = new_train_df['outcome']
X_final_try = new_train_df.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome'])

model_xgb_try = xgb.XGBClassifier(random_state=5)
model_xgb_try.fit(X_final_try, y_final_try)

X_test_try = final_test.drop(columns=['bidder_id', 'payment_account', 'address'])
y_predict_xgb_try = model_xgb_try.predict_proba(X_test_try)[:, 1]
print(len(y_predict_xgb_try))

[16:51:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
4700


In [91]:
y_final_pred = y_predict_catboost*0.8 + y_predict_random_forest*0.1 + y_predict_xgb_try*0.1
y_final_pred

array([0.00215073, 0.00247129, 0.00159892, ..., 0.00082748, 0.00218371,
       0.01211081])

In [92]:
output_df = pd.DataFrame({
    'Id': test_df.bidder_id,
    'Predicted': y_final_pred
})
output_df

,Id,Predicted
0,0013f9b8d1f462df4462e1c1216e441ba6e18,0.002151
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,0.002471
2,002828f800c5132e297cfe3d44fbde9aeac51,0.001599
3,0040cea6b93afd86768c365d89513ffb7c0ba,0.000646
4,00512db4ae953baed983a4bcfa335e7412013,0.021417
...,...,...
4695,ffd49be672b1ba493b07dccf29311045c5392,0.023208
4696,ffdd8ed91a683b0f2a8237798ebe9214e3c43,0.001946
4697,ffde8717e9a547d536a4e9c3f44782829c021,0.000827
4698,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,0.002184


In [93]:
output_df.to_csv('submission6.csv', index=False)